# <font color=red> Data Exploration
#### This notebook aims to find out useful features 
### <font color=blue> Rules :
#### Since we are a lot contributing to the notebook, please comment your code and use explicit variable names
#### Do not modify the raw data as others would like to work on it
#### Only modify your section (copy-paste others code if needed) so that no clashes occur when pulling/pushing on remote repository

## <font color=green>Hugo

In [1]:
# Specific imports in your section

## <font color=green>Nicolas

In [2]:
# Specific imports in your section

## <font color=green>Mohamed

In [62]:
import pandas as pd
import numpy as np
import sklearn as skl
import nltk
from tokenizer import Tokenizer
import seaborn as sns
import time
import sys
from scipy import sparse
sys.path.append('../text_mining')
sys.path.append('../classifier')
from n_gram_creation import vectorize_n_grams
from tfidfvectorizer import TfidfVectorizer
import LogisticRegression

#### Getting the data

In [2]:
train_fname = 'train.csv'
#test_fname = 'test.csv'
X = pd.read_csv(train_fname,header=None)
y = pd.read_csv(train_fname, header=None)[0]
#X_test = pd.read_csv(test_fname, header=None)[0]

In [3]:
X.columns = ['label', 'content']

In [4]:
y.value_counts()

0    3063
1    1352
Name: 0, dtype: int64

In [5]:
index_1 = [index for index in range(X.shape[0]) if y[index]==1]
index_1[0:10]

[6, 8, 11, 12, 15, 18, 23, 27, 28, 29]

#### Some insulting comments

In [6]:
X.content[index_1[:10]] # Wow violent lui ... hahaa

6     "who told you that....stupid ask yourself who ...
8                             "You're an idiot baldo. "
11    "You show your stupidity and ignorance each an...
12    "Wait until you are a minority in your own red...
15    "Its "faggot" you ignorant fuck plus what you ...
18     "Get thatdick outta your mouth so you can talk."
23    "Judging by your prior posts listed in your pr...
27    "RYAN...........GO BACK TO SUCKING YOUR MOTHER...
28                                  "Your are an idiot"
29    "Very sorry I mentioned it.\xa0 Just keep bitc...
Name: content, dtype: object

## Tokenizing Stemming and Cleaning a sentence

In [7]:
# To Keep updated
TOKENS = [
    ### Replacements first ###
    ('BREAK'        , r"\\n", r" "),
    ('SLASH'           , r"\/+", r" "),
    ('WHITE_SPACE'     , r"(\\\\xc2|\\\\xa0|\\xa0|\\xa1|\\xa3|\\xa9|\\r|\\ufeff|\\u2013|\\u2016|\\u200f|\\u2665|\\U0001f308|\\U0001f3e9|\\U0001f48b|\\\\|\\|&nbsp|&amp|=|\"\"|\"|\\u2018|\\u2019)", r" "),
    ('LETTER_A'        , r"(\\xe1|\\xe3|\\xe0|\\xc2|\\u1ef1|\\u0105|\\u1eb7|\\xe5|\\xe4)", r"a"),
    ('LETTER_E'        , r"(\\xe9|\\xe8|\\u1ec3|\\u1ec5|\\u1ebf|\\u0119)", r"e"),
    ('LETTER_I'        , r"(\\xed)", r"i"),
    ('LETTER_O'        , r"(\\x00|\\xf8|\\xf6|\\xf3)", r"o"),
    ('LETTER_U'        , r"(\\xfa|\\xdc|\\xfc|\\u1ee9|\\u0169)", r"u"),
    ('LETTER_Y'        , r"(\\xfd)", r"u"),
    ('LETTER_C'        , r"(\\xe7|\\u0107)", r"c"),
    ('LETTER_Z'        , r"(\\u017a|\\u017e|\\u017c)", r"z"),
    ('PONCT_!'        , r"(\\u203d)", r"!"),
    ### Tokens ###
    ('NOT'             , r"n\'t", r" not "),
    ('PUNCT'      , r"(!+|\?+|\.+)", r" \1 "),
    ('NIL'          , r"\'|,", r" "),
    ('SMILEY'       , r"(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)|xD|XD", r" smiley "),
    ('PUNCT'      , r"(;+|\,+|\)+|\(+)|\-|\_", r" "),
    ('ALONE'        , r"\s[a-hj-zA-HJ-Z]\s", r" "), # delete all lone letters except I&i
    ('DOTS'         , r"\.+", r" "),
#    ('QUOTE'      , r"(\"|\')+(\w+)(\"|\')+", "QUOTE"), # A améliorer pour mettre la citation entre deux <QUOTE>
    ('BEGIN_WHITESPACE' , r"^\s+", r""),
    ('END_WHITESPACE' , r"\s+$", r""),
    ('WHITESPACES' , r"\s+", r" ")
]

# Ca déconne
INSULTS = [
    ### Insults Replacment ###
    ('FUCK'    , r"f(.ck|u.k|uc.)(i?ng*)?", r" fuck "),# fuck/fucking style
    ('FUCK_2'    , r"fuck((\s)?you|.?er)+", r" fuck you "),#fuck you + fuck your XXXer
    ('FUCK_3'    , r"(\w?-)fucker", r" mother fuck "),# compouned fuck # mother fuck plutot ?
    # Replace above regex "mother fuck" by "you fuck" since personal insult ?
    ('ASSHOLE',  r"a.{1,3}oles?", r" asshole "),# ass hole and its versions
    ('STALK',  r"stalk(in*g*|ers?)?", r" stalk "),
    ('STUPID',  r"(\$|s)tupi?d", r" stupid "),
    ('SHIT',  r"sh(itty|it)", r" shit "),
    ('NIGGA',  r" nigg | negro ", r" nigga "),
    ('BEGIN_WHITESPACE' , r"^\s+", r""),
    ('END_WHITESPACE' , r"\s+$", r""),
    ('WHITESPACES' , r"\s+", r" ")
    
    ]

RANDOM = [
    ### Random Replacement ###
    ('LAUGH',  r"\b(?:a*(?:ha)+h?|(?:l+o+)+l+)\b", r" laugh "), # matches laughs such as "hahaha" or "lolll"
    ('PSEUDO',  r"@\s?\w+", r" pseudo "),
    ('DOLLAR',  r"[0-9]+k?\s?\$+", r" dollar "),
    ('DATE',  r"[0-9]+th(\s?of)?(\s\w+)?", r" date "),# eg : 18th or "18th of Month"
    ('YEAR',  r"[0-9]{4}?", r" year "),
    ('NUMBER',  r"[0-9]{1,}", r" number "),
    ('LOVE',  r"<3", r" love "),
    ('URL',  r"(http:)?(\/\/)?((www\.)?\w+\.\w+)\/?(\w+.\w+)?", r" url "),
    ('BEGIN_WHITESPACE' , r"^\s+", r""),
    ('END_WHITESPACE' , r"\s+$", r""),
    ('WHITESPACES' , r"\s+", r" ")
]


In [8]:
myTokenizer = Tokenizer(TOKENS)
myReplacer = Tokenizer(RANDOM)
myReplacer2 = Tokenizer(INSULTS)

In [9]:
input = X.content.values[1055]
print(input)
tokens = myTokenizer.tokenize(input)
new_string = myTokenizer.reconstruct(tokens)
output = myReplacer.tokenize(new_string)
new_string = myReplacer.reconstruct(output)
output = myReplacer2.tokenize(new_string)
print(myReplacer2.reconstruct(output))

"Stick to Santorum.  Your obsession with his wife makes you look like an ass.\\n\\nThere is plenty to pick on with him.  He is as big a freak as you are"
stick to santorum your obsess with his wife make you look like an ass there is plenti to pick on with him he is as big freak as you are


## Tokenizing Stemming and Cleaning the DataSet

In [10]:
X_list = X.content.values

In [11]:
new_X_list = []
for input in X_list:
    tokens = myTokenizer.tokenize(input)
    new_string = myTokenizer.reconstruct(tokens)
    output = myReplacer.tokenize(new_string)
    new_string = myReplacer.reconstruct(output)
    output = myReplacer2.tokenize(new_string)
    new_string = myReplacer2.reconstruct(output)
    new_X_list.append(new_string)

In [12]:
assert len(X_list) == len(new_X_list)

In [13]:
X_new = np.asarray(new_X_list)

In [14]:
X_new.shape

(4415,)

In [15]:
X['processed'] = X_new

In [16]:
X.head()

,label,content,processed
0,0,"""Imagine being able say, you know what, no san...",imagin being abl say you know what no sanction...
1,0,"""""But Jack from Raleigh wasn't done. He came b...",but jack from raleigh was not done he came bac...
2,0,"""the Star box allows you to link your comment ...",the star box allow you to link your comment to...
3,0,"""Cheney,Rush,Nugent.The list is endless.""",cheney rush nugent the list is endless
4,0,"""Obama....I'm blow'n mo smoke up yo arses........",obama i blow mo smoke up yo ar anyon wonder wh...


# <font color=red> TF-IDF

In [17]:
from sklearn import cross_validation

### On définit nos X_train et X_test pour tester nos clssifieurs

In [18]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X.processed, X.label, test_size=0.4, random_state=0)

### Avec Sklearn

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [40]:
count_vect = CountVectorizer(ngram_range=(1,4), analyzer="char")
X_train_counts = count_vect.fit_transform(X_train)

In [27]:
X_train_mycount = vectorize_n_grams(X_train, 4)

In [30]:
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [31]:
my_tf = TfidfVectorizer()
X_train_mytf = my_tf.transform(X_train_mycount[0])

In [32]:
X_train_tf.shape

(2649, 105496)

In [36]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
params = {'C':[0.001, 0.01, 0.1, 1, 5]}
grid = GridSearchCV(logreg, params, cv=3)
grid.fit(X_train_mytf, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 5]}, pre_dispatch='2*n_jobs',
       refit=True, scoring=None, verbose=0)

In [37]:
grid.best_params_, grid.best_score_

({'C': 0.001}, 0.80067950169875424)

## <font color = red> Avec notre TFIDF

### <font color=green>On génère les n-grams

In [26]:
# Bigrams
X_mycount_2, X_dico_2 = vectorize_n_grams(X.processed,2)

In [27]:
# Trigrams
X_mycount_3, X_dico_3 = vectorize_n_grams(X.processed,3)

In [31]:
# Quadrigrams
t0 = time.clock()
X_mycount_4, X_dico_4 = vectorize_n_grams(X.processed,4)
print(time.clock()-t0)

214.192473


In [32]:
# Taille des matrices
X_mycount_2.shape, X_mycount_3.shape, X_mycount_4.shape

((4415, 970), (4415, 8834), (4415, 42622))

In [40]:
np.concatenate((X_dico_2,X_dico_3),axis=0)

array(['ef', 'nb', 'cy', ..., 'pha', 'hyd', 'ja'], 
      dtype='<U3')

In [43]:
# On crée un unique dico
dico_global = np.concatenate((X_dico_2,X_dico_3),axis=0)
dico_global = np.concatenate((dico_global,X_dico_4),axis=0)
dico_global.shape

(52426,)

In [47]:
X_mycount_2.todense()

matrix([[ 0,  0,  0, ...,  3,  1,  0],
        [ 0,  0,  0, ...,  0,  2,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ..., 
        [ 1,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 4,  0,  0, ..., 12,  0,  0]])

In [55]:
# On crée la matrice des counts globale
tmp = np.concatenate((X_mycount_2.todense(), X_mycount_3.todense()), axis=1)
X_new = np.concatenate((tmp, X_mycount_4.todense()), axis=1)

In [56]:
X_new.shape

(4415, 52426)

In [59]:
X_new

matrix([[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 2, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [4, 0, 0, ..., 0, 0, 0]])

In [63]:
X_new_sparse = sparse.csr_matrix(X_new)

In [66]:
X_new_sparse

<4415x52426 sparse matrix of type '<class 'numpy.int64'>'
	with 2697730 stored elements in Compressed Sparse Row format>

### <font color=green> On fait la TFIDF sur cette matrice

In [68]:
my_tfvectorizer = TfidfVectorizer()
X_tf = my_tfvectorizer.transform(X_new_sparse)

In [72]:
X_tf.todense()

matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          3.61363962,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          7.22727924,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 2.45522691,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 9.82090763,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

### <font color = green> On fait notre regression sur cette TFIDF des n_grams

In [74]:
my_logreg = LogisticRegression.LogisticRegression()

#### On définit notre test et notre train

In [75]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_tf, X.label, test_size=0.4, random_state=0)

In [79]:
my_logreg.fit(X_train.todense(), y_train)

ValueError: shapes (2649,) and (1,2649) not aligned: 2649 (dim 0) != 1 (dim 0)

## Classifiers

In [576]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression())
                    ])
                    
_ = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)

In [360]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

          0       0.79      0.96      0.87      1211
          1       0.83      0.45      0.58       555

avg / total       0.80      0.80      0.78      1766



### NAIVE-BAYES : ACCURACY = 0.78

### LOGISTIC REGRESSION = 0.80

## Adding general Features

Addind additional features

In [6]:
class BadWordCounter():
    def __init__(self):
        with open("my_badlist.txt") as f:
            badwords = [l.strip() for l in f.readlines()]
        self.badwords_ = badwords

    def get_feature_names(self):
        return np.array(['n_words', 'n_chars', 'allcaps', 'max_len',
            'mean_len', '@', '!', 'spaces', 'bad_ratio', 'n_bad',
            'capsratio'])

    def fit(self, documents, y=None):
        return self

    def transform(self, documents):
        ## some handcrafted features!
        n_words = [len(c.split()) for c in documents]
        n_chars = [len(c) for c in documents]
        # number of uppercase words
        allcaps = [np.sum([w.isupper() for w in comment.split()])
               for comment in documents]
        # longest word
        max_word_len = [np.max([len(w) for w in c.split()]) for c in documents]
        # average word length
        mean_word_len = [np.mean([len(w) for w in c.split()])
                                            for c in documents]
        # number of google badwords:
        n_bad = [np.sum([c.lower().count(w) for w in self.badwords_])
                                                for c in documents]
        exclamation = [c.count("!") for c in documents]
        addressing = [c.count("@") for c in documents]
        spaces = [c.count(" ") for c in documents]

        allcaps_ratio = np.array(allcaps) / np.array(n_words, dtype=np.float)
        bad_ratio = np.array(n_bad) / np.array(n_words, dtype=np.float)

        return np.array([n_words, n_chars, allcaps, max_word_len,
            mean_word_len, exclamation, addressing, spaces, bad_ratio, n_bad,
            allcaps_ratio]).T

In [5]:
bad_counter = BadWordCounter()

In [6]:
features = bad_counter.transform(X.content)

In [7]:
features.shape

(4415, 11)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score

In [22]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, y, test_size=0.4, random_state=0)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf_1 = SVC(kernel='linear', C=1)
clf_2 = LinearRegression()
cross_validation.cross_val_score(clf_1, features, y, cv=5)